## PT DDP Launcher Testing
This notebook tests the following combination:

* image: PT training DLC with my changes
* distribution = pytorchddp, backend = nccl

In [20]:
#!pip uninstall -y sagemaker

In [31]:
%%time
! python3 -m pip install --upgrade sagemaker


CPU times: user 15.9 ms, sys: 9.22 ms, total: 25.1 ms
Wall time: 2.26 s


In [1]:
%pip show sagemaker

Name: sagemaker
Version: 2.110.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, numpy, packaging, pandas, pathos, protobuf, protobuf3-to-dict, schema, smdebug-rulesconfig
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sagemaker
import boto3

beta_endpoint = 'https://sagemaker.beta.us-west-2.ml-platform.aws.a2z.com'
beta_runtime_endpoint = 'https://maeveruntime.loadtest.us-west-2.ml-platform.aws.a2z.com'
sage = boto3.client('sagemaker', endpoint_url=beta_endpoint)
sage_runtime = boto3.client('sagemaker-runtime', endpoint_url=beta_runtime_endpoint)
sess = sagemaker.Session(sagemaker_client=sage, sagemaker_runtime_client=sage_runtime)

role = sagemaker.get_execution_role()


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
#Add instructions for local environment later, if needed

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


sagemaker role arn: arn:aws:iam::570106654206:role/viskaria-dev
sagemaker bucket: sagemaker-us-west-2-570106654206
sagemaker session region: us-west-2


In [65]:
#Use DLC image which has GDR Copy library installed + EFA version upgraded to 1.18.0
image_uri = '570106654206.dkr.ecr.us-west-2.amazonaws.com/viskaria-dev:gdr-installed'

estimator = PyTorch(
    base_job_name="gdr-driver",
    source_dir="./code/",
    entry_point="install.py",
    role=role,
    py_version="py38",
    framework_version="1.12",
    image_uri=image_uri,
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.g5.8xlarge",
    sagemaker_session=sess,
    # Training using SMDataParallel Distributed Training Framework
    debugger_hook_config=False,
)
estimator.fit()

2022-10-10 23:13:18 Starting - Starting the training job...
2022-10-10 23:13:34 Starting - Preparing the instances for trainingProfilerReport-1665443598: InProgress
......
2022-10-10 23:14:45 Downloading - Downloading input data...
2022-10-10 23:15:18 Training - Downloading the training image...........................
2022-10-10 23:19:42 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-10 23:19:45,702 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-10 23:19:45,722 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-10 23:19:45,728 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-10 23:19:46,301 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "chann

In [66]:
image_uri = '570106654206.dkr.ecr.us-west-2.amazonaws.com/viskaria-dev:gdr-installed'

estimator = PyTorch(
    base_job_name="gdr-driver",
    source_dir="./code/",
    entry_point="install.py",
    role=role,
    py_version="py38",
    framework_version="1.12",
    image_uri=image_uri,
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sess,
    # Training using SMDataParallel Distributed Training Framework
    debugger_hook_config=False,
)
estimator.fit()

2022-10-10 23:21:16 Starting - Starting the training job...
2022-10-10 23:21:24 Starting - Insufficient capacity error from EC2 while launching instances, retrying!ProfilerReport-1665444076: InProgress
...
2022-10-10 23:22:16 Failed - Training job failed
..

CapacityError: Error for Training job gdr-driver-2022-10-10-23-21-16-061: Failed. Reason: CapacityError: Unable to provision requested ML compute capacity. Please retry using a different ML instance type.

In [69]:
### Ignore this cell - scratch implementation

# Use SM DLC directly
estimator = PyTorch(
    base_job_name="gdr-driver",
    source_dir="./code/",
    entry_point="install.py",
    role=role,
    py_version="py38",
    framework_version="1.12",
#     image_uri=image_uri,
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.g5.8xlarge",
    sagemaker_session=sess,
    # Training using SMDataParallel Distributed Training Framework
    debugger_hook_config=False,
)
estimator.fit()

2022-10-11 00:13:47 Starting - Starting the training job...
2022-10-11 00:14:11 Starting - Preparing the instances for trainingProfilerReport-1665447226: InProgress
......
2022-10-11 00:15:13 Downloading - Downloading input data...
2022-10-11 00:15:31 Training - Downloading the training image........................
2022-10-11 00:19:45 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-11 00:19:47,769 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-11 00:19:47,790 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-11 00:19:47,795 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-11 00:19:48,339 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_